# NoSQL (HBase) (sesión 5)

![Image of HBase](http://hbase.apache.org/images/hbase_logo_with_orca_large.png)

Esta hoja muestra cómo acceder a bases de datos HBase y también a conectar la salida con Jupyter.

Se puede utilizar el *shell* propio de HBase en la máquina virtual llamando al programa:

    $ hbase/bin/hbase shell

La diferencia es que ese programa espera código Ruby y aquí trabajaremos con Python.

### Nota sobre la caída de RegionServers

En este entorno con poca memoria son frecuentes las caídas de RegionServers. Sería conveniente:

- dar a la memoria virtual al menos 3GB de memoria,
- aumentar el tamaño del HEAP de los procesos de HBase, y
- aumentar el tiempo de _timeout_ de Zookeeper.

En la [documentación de HBase](http://hbase.apache.org/book.html#trouble.rs) dan unas recomendaciones, sobre todo, para carga inicial, como he realizado estos días para cargar las bases de datos de ejemplo:

> Make sure you give plenty of RAM (in hbase-env.sh), the default of 1GB won’t be able to sustain long running imports.
>
> [...]
>
> If this is happening during an upload which only happens once (like initially loading all your data into HBase), consider bulk loading.

Aunque no usamos _bulk loading_ para mostrar cómo se añaden datos desde Python (el _bulk loading_ hay que hacerlo en Java).

Las caídas en los RegionServers pueden producirse por varias cuestiones: falta de memoria, timeout por la ejecución del GC de Java, etc. Estas caídas son aceptadas como normales por el sistema HBase, que continuará funcionando con el resto de RegionServers y aceptará un RegionServer que terminó abruptamente una vez reiniciado.

En nuestra VM sólo hay un RegionServer, y se puede iniciar si cayó con el comando:

    ~/hbase/bin/start-daemon.sh start regionserver

El siguiente _script_ comprueba cada 30 segundos la salida de depuración del Máster de HBase, y si ve que no hay RegionServers, llama al script de reinicio del único RegionServer. El cliente continuará sin problemas después de unos segundos de inicialización. Al cabo del tiempo, los RegionServer que no funcionan se eliminan por HBase.

In [ ]:
%%writefile restart-regionserver.sh
#! /bin/sh
while true ; do
	ns=`curl -s http://localhost:60010/jmx | grep numRegionServers | tr -cd [0-9]`
	test -z "$ns" || test $ns -gt 0 || ~/hbase/bin/hbase-daemon.sh start regionserver 
	sleep 30 
done


In [ ]:
!chmod a+x restart-regionserver.sh

In [ ]:
# Config
%env DIR=/vagrant

In [ ]:
!~/start-hbase.sh

In [ ]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [ ]:
import happybase

host = '127.0.0.1'
connection = happybase.Connection(host)
connection.tables()

In [ ]:
%%bash
(test -e $DIR/Posts.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Posts.csv.gz -O - 2>/dev/null | gunzip > $DIR/Posts.csv) \
  && echo OK)

In [ ]:
%%bash
~/hadoop/bin/hadoop fs -copyFromLocal $DIR/Posts.csv hdfs:///

In [ ]:
%%bash
~/hbase/bin/hbase org.apache.hadoop.hbase.mapreduce.ImportTsv -Dimporttsv.separator=, -Dimporttsv.columns=HBASE_ROW_KEY,test:ParentId,test:Body,test:ViewCount,test:LastEditorDisplayName,test:ClosedDate,test:Title,test:LastEditorUserId,test:LastActivityDate,test:LastEditDate,test:AnswerCount,test:CommentCount,test:AcceptedAnswerId,test:Score,test:OwnerDisplayName,test:PostTypeId,test:OwnerUserId,test:Tags,test:CreationDate,test:FavoriteCount,test:CommunityOwnedDate posts hdfs:///Posts.csv

In [ ]:
%%bash
(test -e $DIR/Users.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Users.csv.gz -O - 2>/dev/null | gunzip > $DIR/Users.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e $DIR/Tags.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Tags.csv.gz -O - 2>/dev/null | gunzip > $DIR/Tags.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e $DIR/Comments.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Comments.csv.gz -O - 2>/dev/null | gunzip > $DIR/Comments.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e $DIR/Votes.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Votes.csv.gz -O - 2>/dev/null | gunzip > $DIR/Votes.csv) \
  && echo OK)

## EJERCICIO: Con HBase, mostrar las consultas RQ1 a RQ4 de la sesión 2